## FINANCIAL DATA

MODULE 3 | LESSON 1


---


# CALCULATING PORTFOLIO RISK/RETURN STATISTICS

|  |  |
|:---|:---|
|**Reading Time** |  40 minutes |
|**Prior Knowledge** | Simple Stock Returns, Variance, Python, Linear Algebra, Matrix Multiplication  |
|**Keywords** | Portfolio Return, Variance, Sharpe Ratio |

---

*The previous module had a focus on the individual security. For this lesson, we take what we have learned previously and apply it to a portfolio setting using a basket of stocks/crypto.*

In [ ]:
import datetime
import math

import numpy as np
import pandas_datareader.data as web
import yfinance as yfin
from IPython.display import VimeoVideo

yfin.pdr_override()

## 1. Portfolio Returns
Up to this point in the course, we've spent our time analyzing individual securities. Today, we perform a similar analysis except using a portfolio of assets. We will first quickly recap how to determine return on investment for a single asset, General Electric (GE).

### 1.1 Single Asset Return Recap
We will assume we bought 100 shares 10 years ago. To determine the cash return, we just need to use the following formula:

$r_{_f} = 100\ (p_{_f} - p_{_i})$

In [ ]:
# Define all initial variables
# start = datetime.date.today()-datetime.timedelta(365*10)
# end = datetime.date.today()
end_GE = datetime.date(2022, 12, 31)
start_GE = end_GE - datetime.timedelta(365 * 10)

price_GE = web.DataReader(["GE"], start_GE, end_GE)["Adj Close"]
initialPrice_GE = price_GE[0]
finalPrice_GE = price_GE[-1]
cashReturn_GE = (finalPrice_GE - initialPrice_GE) * 100

print(
    "With an initial investment of $%.2f, \
    \nthe cash return of this investment would be \
    \n($%.3f - $%.3f) * 100 = $%.3f"
    % (initialPrice_GE * 100, finalPrice_GE, initialPrice_GE, cashReturn_GE)
)

### 1.2 How to Calculate Return with a Basket of Assets
We've now gone over returns of a single asset many times during this course. It's time to apply similar logic to a basket of multiple assets. This will once again be best illustrated with an example. The following calculations can easily be extended to n number of securities, but to keep it simple, we will use two for our example. For this, we need to make some basic assumptions:

* 2 stocks
    * Meta—We will refer to it as its previous name from here on out: Facebook (META)
    * Chipotle (CMG)
* Bought 100 shares of each five years ago
* Goal: Calculate percentage return obtained by the portfolio at the end of the period

To start, we need to calculate the weights of each asset at the start of the period:

In [ ]:
# Define all initial variables
# start = datetime.date.today()-datetime.timedelta(365*5)
# end = datetime.date.today()
start_stocks = datetime.date(2016, 11, 29)
end_stocks = datetime.date(2021, 11, 28)

prices_stocks = web.DataReader(["META", "CMG"], start_stocks, end_stocks)["Adj Close"]
initialMETA = prices_stocks.META[0]
initialCMG = prices_stocks.CMG[0]
finalMETA = prices_stocks.META[-1]
finalCMG = prices_stocks.CMG[-1]
METAWeight = initialMETA / (initialMETA + initialCMG)
CMGWeight = initialCMG / (initialMETA + initialCMG)

In [ ]:
print(
    "We have an initial investment in META of $%.2f and in CMG $%.2f"
    % (initialMETA * 100, initialCMG * 100)
)

In [ ]:
print(
    "This would make the weights %.2f and %.2f for META and CMG respectively"
    % (METAWeight, CMGWeight)
)

### 1.3 Final Returns


In order to calculate the final portfolio percentage returns, we need to find the returns of each asset individually, multiply the return by the weight in our portfolio, and then add them together. The formula to calculate this for two assets is:

$\text{Portfolio Return} = w_1 R_1 + w_2 R_2$

In [ ]:
returnMETA = 100 * (finalMETA - initialMETA) / initialMETA
returnCMG = 100 * (finalCMG - initialCMG) / initialCMG

In [ ]:
print(
    "This return over this period for Facebook is %.2f%% and %.2f%% for Chipotle"
    % (returnMETA, returnCMG)
)

In [ ]:
print(
    "Multiplying these individual returns by their weights gives %.2f (META) and %.2f (CMG)"
    % (returnMETA * METAWeight, returnCMG * CMGWeight)
)

In [ ]:
returnMETA * METAWeight + returnCMG * CMGWeight

Adding these weighted returns together gives us a portfolio return of 41.14% + 250.35% = 291.49%

Our portfolio would have returned 291.49% over the last five years, assuming we invested in both assets on the same starting date and with our weights.

In the first video of this lesson, we show how to transition from calculating an individual security's returns to calculating the returns of a portfolio of assets.<span style='color: transparent; font-size:1%'>All rights reserved WQU WorldQuant University QQQQ</span>

In [ ]:
VimeoVideo("706655699", h="1a478ba2bf", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/1XBLrPLm9xtBfS74KK2xve2TuU5Ci9ZOf/view?usp=sharing)

## 2. Calculating Portfolio Variance
This was discussed at a high level during the Financial Markets course, but here, we will show how to calculate variance in Python with empirical data. While returns are important, investors are also concerned about risk or volatility. We will use variance of returns to determine risk. Once again, we will use an example, using the same data as before to drive the point home. A bit of linear algebra is needed, but luckily, we have Python to do the tough calculations for us.

If we were to calculate this for two assets using pencil and paper, we could use the following formula:

$$\textrm{Portfolio variance} = w_1^2 \sigma_1^2 + w_2^2\sigma_2^2 + 2 w_1 w_2 Cov_{1,2}$$

Where:

$w_1$ = the portfolio weight of the first asset  </br>
$w_2$ = the portfolio weight of the second asset  </br>
$\sigma_1$ = the standard deviation of the first asset  </br>
$\sigma_2$ = the standard deviation of the second asset  </br>
$Cov_{1,2}$ = the covariance of the two assets, which can thus be expressed as $\rho_{_{(1,2)}} \sigma_1 \sigma_2$, where $\rho_{_{(1,2)}}$ is the correlation coefficient between the two assets  </br>


Luckily, Python makes it easy for us to expand the formula to $n$ number of assets. This formula is better expressed in matrix notation since it's easier to apply this way in Python:

$$\text{Portfolio Variance} = \text{Weights transposed} \times (\text{covariance matrix}) \times \text{weights}$$



### 2.1 Defining Variables

Let's put those portfolio weights we calculated before into an array and also calculate the simple daily returns of our asset along with a covariance matrix of these daily returns. Keep in mind that we will multiply the daily returns covariance matrix by 252 to measure annual variance.


In [ ]:
weights_stocks = np.array([0.77, 0.23])
returns_stocks = prices_stocks.pct_change()
covariance_stocks = 252 * returns_stocks.cov()

After defining these, we simply need to take the dot product of the transposed weights with the dot product of the covariance matrix and weights to get the annual variance of our portfolio. Note: The t-method in numpy just transposes the array.

In [ ]:
variance_stocks = np.dot(weights_stocks.T, np.dot(covariance_stocks, weights_stocks))

# Print the result
print(f"The portfolio variance is {np.round(variance_stocks, 4)}")

We determine that the annual variance of our portfolio is 0.1033. Let's compare this to the variance of each individual asset.

### 2.2 Comparing Portfolio Variance to Individual Stock Variance
This shows the annual variance of a portfolio with META and CMG weighted 23% and 77% respectively has shown a 0.1033 annual variance over the last five years.

One extremely interesting byproduct of this result can be seen when calculating variance of the equities individually. One of the main pillars of modern portfolio theory is about how diversification reduces risk. We can see this clear as day by looking at each stock's variance over the same time period.

In [ ]:
returns_stocks.var() * 252

We can see in above that both stocks' variance is higher than that of our portfolio, and even though CMG has a 77% weighting and a 13.7% variance, our portfolio's variance is more than 5 percentage points lower. This is because, if assets are not perfectly correlated, there will be some diversification benefit to risk by investing in multiple securities.

### 2.3 Calculating Standard Deviation of the Portfolio
Taking this a step further, since we know standard deviation is just the square root of variance, we can also calculate that here.

The below calculation shows the annual standard deviation of our portfolio to be about 32%:

In [ ]:
np.round(math.sqrt(variance_stocks) * 100, 2)

Just like with the last video where we moved from thinking about individual stocks to a basket of stocks, we do the same here in order to calculate variance and standard deviation of returns for a portfolio.

In [ ]:
VimeoVideo("706655745", h="8aa2f6d895", width=600)

##### [Access video transcript here](https://drive.google.com/file/d/1GJfJMRmDvKy1Hd8eUWL8VeGQcuwAsR-I/view?usp=sharing)

## 3. Sharpe Ratio of Portfolio
Calculating the Sharpe ratio of a portfolio is pretty easy at this point since we have all the variables we need at our disposal. As a recap, the Sharpe ratio is just:

$$\frac{\text{Portfolio Return} - \text{risk-free rate}}{\text{Portfolio Standard Dev}}$$

Since interest rates have been near 0 for a while now, we will assume 0 for the risk-free rate. Our formula is now:

$$\frac{\text{Portfolio Return}}{\text{Portfolio Standard Dev}}$$

We have calculated the total return of our portfolio over five years to be 291.49%. To get a single-year return, we can divide this number by 5 to get 58.298%. Considering our annual standard deviation is 32.14%, we can get the Sharpe ratio by dividing these two numbers to get 1.8138 as the Sharpe ratio for our portfolio. In most cases, a Sharpe ratio over 1 is an exceptional risk/return metric. This makes sense considering how advantageous investments in Chipotle or Facebook would have been if you had invested in them five years ago.


## 4. Performing Similar Analysis on Cryptocurrencies

We will use the two most popular cryptocurrencies to perform a similar analysis, and we will compare and contrast them to the equity results we have obtained. We'll perform the analysis over the same time frame (the last five years) for the two most popular cryptocurrencies: Bitcoin and Ethereum. This time, let's say we bought five Bitcoin and 100 Ethereum five years ago.


In [ ]:
# Define all initial variables
# start = datetime.date.today()-datetime.timedelta(365*5)
# end = datetime.date.today()

start_crypto = datetime.date(2017, 11, 29)
end_crypto = datetime.date(2022, 11, 28)

prices_crypto = web.DataReader(["BTC-USD", "ETH-USD"], start_crypto, end_crypto)[
    "Adj Close"
]
prices_crypto = prices_crypto.rename(
    columns={"BTC-USD": "Bitcoin", "ETH-USD": "Ethereum"}
)

initialBTC = 5 * prices_crypto.Bitcoin[0]
initialETH = 100 * prices_crypto.Ethereum[0]
finalBTC = 5 * prices_crypto.Bitcoin[-1]
finalETH = 100 * prices_crypto.Ethereum[-1]
BTCWeight = initialBTC / (initialBTC + initialETH)
ETHWeight = initialETH / (initialBTC + initialETH)

In [ ]:
print(
    "This would make the weights %.2f and %.2f for Bitcoin and Ethereum respectively"
    % (BTCWeight, ETHWeight)
)

In [ ]:
returnBTC = 100 * (finalBTC - initialBTC) / initialBTC
returnETH = 100 * (finalETH - initialETH) / initialETH

In [ ]:
np.round(returnBTC, 5)

In [ ]:
np.round(returnETH, 5)

Bitcoin returned 66.30% and Ethereum has returned 179.55%. If we keep using our portfolio weights from before, our portfolio would have returned 118.81%.

In [ ]:
np.round(returnBTC * BTCWeight + returnETH * ETHWeight, 5)

### 4.1 Calculating Variance for Cryptos
Notice instead of multiplying by 252 like we did previously to annualize our variance, we use 365 here since cryptos trade seven days a week:

In [ ]:
# Calculating portfolio variance
weights_crypto = np.array([BTCWeight, ETHWeight])
returns_crypto = prices_crypto.pct_change()
covariance = 365 * returns_crypto.cov()
variance_crypto = np.dot(weights_crypto.T, np.dot(covariance, weights_crypto))
print(
    f"The variance of the portfolio consisting of BTC and ETH is: {np.round(variance_crypto, 3)}"
)

We get 0.661 variance for our portfolio, which translates to an annual standard deviation of 81.30%. Even though we were in relatively risky stocks, the standard deviation of that portfolio (32.14%) pales in comparison to this.

Lets now calculate the Sharpe ratio.

### 4.2 Sharpe Ratio Comparison
The Sharpe ratio of our stock portfolio was 1.8138. If we run the same calculation on our crypto portfolio we get:

118.81 / 5 = 23.76% as our annual percentage return. If we divide this by the standard deviation we get:

23.76% / 81.30% = 0.2922 Sharpe Ratio

We observe that the Sharpe Ratio of the Crypto portfolio is lower than the one of the stocks. This is somewhat different from what we got used to reading in the news and blogs concerning the amazing returns of the crypto assets in general. Of course, the choice of the 5 years is very important since the volatility and price swings throughout this period are much greater. For example, if the end date for the crypto portfolio was chosen at the top of the bull market, the Sharpe ratio would have been a lot greater.

## 5. Conclusion

During this lesson, we hint and imply that diversifying reduces risk in a portfolio. In the next lesson, we will prove it by quantifying the benefits of diversification.

---
Copyright 2023 WorldQuant University. This
content is licensed solely for personal use. Redistribution or
publication of this material is strictly prohibited.
